# 🏍️ Vintage Bike Video Generator

**Low-Compute GPU Video Generation with Streamlit Frontend**

This notebook generates context-specific videos for vintage motorcycles using optimized, low-compute video generation models.

### Features:
- 🎬 Text-to-Video generation optimized for Colab's free GPU
- 🏍️ Context-aware prompting for vintage bikes
- 🎨 Streamlit frontend for easy interaction
- ⚡ Memory-efficient with FP16 and attention slicing

---

## 1️⃣ Install Dependencies

In [ ]:
# Install required packages
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers transformers accelerate
!pip install -q streamlit pyngrok imageio[ffmpeg] opencv-python-headless
!pip install -q safetensors xformers

print("✅ All dependencies installed!")

## 2️⃣ Check GPU Availability

In [ ]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU Available: {gpu_name}")
    print(f"📊 GPU Memory: {gpu_memory:.2f} GB")
else:
    print("❌ No GPU detected! Please enable GPU in Runtime > Change runtime type > GPU")

## 3️⃣ Create the Streamlit App Files

In [ ]:
%%writefile video_config.py
"""Configuration for Vintage Bike Video Generator"""

# Model Configuration - Using ModelScope (lowest compute requirement)
MODEL_CONFIG = {
    "model_id": "damo-vilab/text-to-video-ms-1.7b",  # ~3.4GB, works on T4 GPU
    "torch_dtype": "float16",
    "variant": "fp16"
}

# Video Generation Settings (optimized for low compute)
VIDEO_CONFIG = {
    "num_frames": 16,       # Keep low for faster generation
    "height": 256,          # Lower resolution = faster
    "width": 256,           # Lower resolution = faster
    "num_inference_steps": 25,  # Lower = faster but less quality
    "guidance_scale": 7.5,
    "fps": 8
}

# Vintage Bike Context Keywords
ALLOWED_KEYWORDS = {
    "vintage", "classic", "motorcycle", "bike", "cafe racer",
    "retro", "1960s", "1970s", "1980s", "chrome", "air cooled",
    "triumph", "norton", "bsa", "harley", "indian", "royal enfield",
    "chopper", "bobber", "scrambler", "custom", "restoration"
}

BLOCKED_KEYWORDS = {
    "person", "people", "face", "animal", "gun", "weapon",
    "car", "truck", "futuristic", "cyberpunk", "modern",
    "robot", "alien", "fantasy", "nude", "violence"
}

# Era Descriptions
ERA_DESCRIPTIONS = {
    "1950s": "1950s vintage motorcycle, classic chrome details, leather seat, spoke wheels",
    "1960s": "1960s vintage motorcycle, British cafe racer, analog gauges, clip-on handlebars",
    "1970s": "1970s vintage motorcycle, cafe racer style, chrome exhaust pipes, racing stripes",
    "1980s": "1980s retro motorcycle, air cooled engine, classic bodywork, period-correct colors",
    "Custom": "custom vintage motorcycle, hand-built, unique details, artisan craftsmanship"
}

# Camera Styles
CAMERA_STYLES = {
    "Static": "static tripod shot, steady frame, professional photography",
    "Pan": "slow cinematic pan, smooth horizontal movement, revealing shot",
    "Tracking": "smooth tracking shot, following the motorcycle, dynamic movement",
    "Orbit": "orbiting camera movement, 360 degree view, showcase shot",
    "Close-up": "close-up detail shot, macro view, highlighting craftsmanship"
}

# Base Style Prompt
BASE_STYLE = (
    "cinematic shot, realistic motion, professional cinematography, "
    "natural lighting, shallow depth of field, warm vintage color grading, "
    "film grain texture, 24fps, high quality"
)

print("✅ Configuration loaded!")

In [ ]:
%%writefile video_engine.py
"""Video Generation Engine - Optimized for Low Compute"""

import torch
import gc
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import numpy as np
from video_config import MODEL_CONFIG, VIDEO_CONFIG

class VintageVideoGenerator:
    """Low-compute video generator optimized for Colab's T4 GPU"""
    
    def __init__(self):
        self.pipe = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
    def load_model(self):
        """Load model with memory optimizations"""
        if self.pipe is not None:
            return
        
        print("🔄 Loading video generation model...")
        print(f"📍 Using device: {self.device}")
        
        # Clear GPU memory first
        torch.cuda.empty_cache()
        gc.collect()
        
        # Load with FP16 for memory efficiency
        self.pipe = DiffusionPipeline.from_pretrained(
            MODEL_CONFIG["model_id"],
            torch_dtype=torch.float16,
            variant="fp16"
        )
        
        # Use faster scheduler
        self.pipe.scheduler = DPMSolverMultistepScheduler.from_config(
            self.pipe.scheduler.config
        )
        
        # Memory optimizations
        self.pipe.enable_model_cpu_offload()  # Offload to CPU when not in use
        self.pipe.enable_vae_slicing()        # Process VAE in slices
        
        # Try to enable xformers for even more memory efficiency
        try:
            self.pipe.enable_xformers_memory_efficient_attention()
            print("✅ xFormers memory efficient attention enabled")
        except:
            print("⚠️ xFormers not available, using default attention")
        
        print("✅ Model loaded successfully!")
        
    def generate(self, prompt: str, seed: int = None) -> list:
        """Generate video frames from prompt"""
        self.load_model()
        
        # Clear memory before generation
        torch.cuda.empty_cache()
        gc.collect()
        
        # Set seed for reproducibility
        if seed is None:
            seed = torch.randint(0, 2**32, (1,)).item()
        
        generator = torch.Generator(device="cuda").manual_seed(seed)
        
        print(f"🎬 Generating video with seed: {seed}")
        print(f"📝 Prompt: {prompt[:100]}...")
        
        # Generate video
        with torch.inference_mode():
            output = self.pipe(
                prompt=prompt,
                num_frames=VIDEO_CONFIG["num_frames"],
                height=VIDEO_CONFIG["height"],
                width=VIDEO_CONFIG["width"],
                num_inference_steps=VIDEO_CONFIG["num_inference_steps"],
                guidance_scale=VIDEO_CONFIG["guidance_scale"],
                generator=generator
            )
        
        # Get frames as numpy arrays
        frames = output.frames[0]  # Shape: (num_frames, H, W, 3)
        
        # Convert to list of numpy arrays
        if isinstance(frames, np.ndarray):
            frame_list = [frames[i] for i in range(frames.shape[0])]
        else:
            # If it's PIL images
            frame_list = [np.array(f) for f in frames]
        
        print(f"✅ Generated {len(frame_list)} frames")
        
        # Clear memory after generation
        torch.cuda.empty_cache()
        gc.collect()
        
        return frame_list, seed
    
    def unload(self):
        """Unload model to free memory"""
        if self.pipe is not None:
            del self.pipe
            self.pipe = None
            torch.cuda.empty_cache()
            gc.collect()
            print("✅ Model unloaded")

print("✅ Video engine module loaded!")

In [ ]:
%%writefile prompt_builder.py
"""Prompt Builder for Vintage Bike Context"""

from video_config import (
    ERA_DESCRIPTIONS, CAMERA_STYLES, BASE_STYLE,
    ALLOWED_KEYWORDS, BLOCKED_KEYWORDS
)

def validate_prompt(prompt: str) -> tuple:
    """
    Validate prompt for vintage bike context
    Returns: (is_valid, message)
    """
    p = prompt.lower().strip()
    
    if not p:
        return False, "❌ Please enter a prompt"
    
    # Check for blocked keywords
    for blocked in BLOCKED_KEYWORDS:
        if blocked in p:
            return False, f"❌ Content not allowed: '{blocked}' is blocked"
    
    # Check for at least one allowed keyword (encourage vintage bike content)
    has_context = any(allowed in p for allowed in ALLOWED_KEYWORDS)
    
    if not has_context:
        return False, "❌ Please include vintage motorcycle context (e.g., 'vintage motorcycle', 'classic bike', 'cafe racer')"
    
    return True, "✅ Prompt validated"


def build_prompt(user_prompt: str, era: str, camera: str, scene: str = "") -> str:
    """
    Build enhanced prompt with context
    """
    # Get era description
    era_desc = ERA_DESCRIPTIONS.get(era, ERA_DESCRIPTIONS["1970s"])
    
    # Get camera style
    camera_desc = CAMERA_STYLES.get(camera, CAMERA_STYLES["Pan"])
    
    # Build the enhanced prompt
    parts = [
        camera_desc,
        era_desc,
        user_prompt.strip(),
    ]
    
    if scene:
        parts.append(scene)
    
    parts.append(BASE_STYLE)
    
    # Combine all parts
    final_prompt = ", ".join(parts)
    
    return final_prompt


# Preset prompts for quick generation
PRESET_PROMPTS = {
    "🏍️ Classic Cafe Racer": "classic cafe racer motorcycle parked on cobblestone street, morning light, steam rising",
    "🔧 Garage Scene": "vintage motorcycle in rustic garage workshop, tools on workbench, oil can, warm lighting",
    "🌅 Sunset Ride": "vintage motorcycle silhouette against golden sunset, desert highway, epic cinematic",
    "🏁 Racing Heritage": "classic racing motorcycle on vintage race track, checkered flag visible, motion blur",
    "🪞 Chrome Details": "close-up of vintage motorcycle chrome engine, reflections, polished metal, detailed",
    "🌿 Country Road": "vintage motorcycle on winding country road, autumn leaves, peaceful scenic route"
}

print("✅ Prompt builder module loaded!")

In [ ]:
%%writefile video_utils.py
"""Video Utilities - Export and Caching"""

import os
import hashlib
import imageio
import numpy as np
from typing import List
from video_config import VIDEO_CONFIG

# Cache directory
CACHE_DIR = "generated_videos"
os.makedirs(CACHE_DIR, exist_ok=True)


def frames_to_video(frames: List[np.ndarray], output_path: str, fps: int = None) -> str:
    """
    Convert frames to MP4 video
    """
    if fps is None:
        fps = VIDEO_CONFIG["fps"]
    
    # Ensure frames are uint8
    processed_frames = []
    for frame in frames:
        if frame.dtype != np.uint8:
            if frame.max() <= 1.0:
                frame = (frame * 255).astype(np.uint8)
            else:
                frame = frame.astype(np.uint8)
        processed_frames.append(frame)
    
    # Save video
    imageio.mimsave(
        output_path,
        processed_frames,
        fps=fps,
        codec="libx264",
        quality=8
    )
    
    return output_path


def frames_to_gif(frames: List[np.ndarray], output_path: str, fps: int = None) -> str:
    """
    Convert frames to GIF
    """
    if fps is None:
        fps = VIDEO_CONFIG["fps"]
    
    # Ensure frames are uint8
    processed_frames = []
    for frame in frames:
        if frame.dtype != np.uint8:
            if frame.max() <= 1.0:
                frame = (frame * 255).astype(np.uint8)
            else:
                frame = frame.astype(np.uint8)
        processed_frames.append(frame)
    
    # Save GIF
    imageio.mimsave(
        output_path,
        processed_frames,
        fps=fps,
        loop=0
    )
    
    return output_path


def generate_cache_key(prompt: str, era: str, camera: str, seed: int) -> str:
    """
    Generate cache key for prompt
    """
    key_string = f"{prompt}_{era}_{camera}_{seed}"
    return hashlib.sha256(key_string.encode()).hexdigest()[:16]


def get_cached_video(cache_key: str) -> str:
    """
    Get cached video if exists
    """
    mp4_path = os.path.join(CACHE_DIR, f"{cache_key}.mp4")
    if os.path.exists(mp4_path):
        return mp4_path
    return None


def save_video(frames: List[np.ndarray], cache_key: str) -> tuple:
    """
    Save video to cache and return paths
    """
    mp4_path = os.path.join(CACHE_DIR, f"{cache_key}.mp4")
    gif_path = os.path.join(CACHE_DIR, f"{cache_key}.gif")
    
    frames_to_video(frames, mp4_path)
    frames_to_gif(frames, gif_path)
    
    return mp4_path, gif_path


def list_cached_videos() -> list:
    """
    List all cached videos
    """
    videos = []
    for f in os.listdir(CACHE_DIR):
        if f.endswith(".mp4"):
            videos.append(os.path.join(CACHE_DIR, f))
    return sorted(videos, key=os.path.getmtime, reverse=True)

print("✅ Video utilities module loaded!")

In [ ]:
%%writefile streamlit_app.py
"""
🏍️ Vintage Bike Video Generator - Streamlit Frontend
A context-specific video generation app for vintage motorcycles
"""

import streamlit as st
import os
import time
from datetime import datetime

# Import our modules
from video_config import ERA_DESCRIPTIONS, CAMERA_STYLES, VIDEO_CONFIG
from prompt_builder import validate_prompt, build_prompt, PRESET_PROMPTS
from video_utils import save_video, get_cached_video, generate_cache_key, list_cached_videos
from video_engine import VintageVideoGenerator

# Page Configuration
st.set_page_config(
    page_title="🏍️ Vintage Bike Video Generator",
    page_icon="🏍️",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for vintage aesthetic
st.markdown("""
<style>
    /* Main background */
    .stApp {
        background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%);
    }
    
    /* Headers */
    h1, h2, h3 {
        color: #e8d5b7 !important;
        font-family: 'Georgia', serif !important;
    }
    
    /* Main title styling */
    .main-title {
        text-align: center;
        font-size: 3rem;
        background: linear-gradient(90deg, #d4a574, #e8d5b7, #d4a574);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
        margin-bottom: 0.5rem;
    }
    
    .subtitle {
        text-align: center;
        color: #a8a8a8 !important;
        font-size: 1.1rem;
        margin-bottom: 2rem;
    }
    
    /* Cards */
    .stCard {
        background: rgba(255, 255, 255, 0.05);
        border-radius: 15px;
        padding: 1.5rem;
        border: 1px solid rgba(212, 165, 116, 0.3);
    }
    
    /* Buttons */
    .stButton > button {
        background: linear-gradient(135deg, #d4a574 0%, #c49058 100%);
        color: #1a1a2e;
        font-weight: bold;
        border: none;
        border-radius: 10px;
        padding: 0.75rem 2rem;
        transition: all 0.3s ease;
    }
    
    .stButton > button:hover {
        transform: translateY(-2px);
        box-shadow: 0 5px 20px rgba(212, 165, 116, 0.4);
    }
    
    /* Sidebar */
    .css-1d391kg {
        background: rgba(26, 26, 46, 0.95);
    }
    
    /* Text inputs */
    .stTextArea textarea {
        background: rgba(255, 255, 255, 0.05);
        border: 1px solid rgba(212, 165, 116, 0.3);
        color: #e8d5b7;
        border-radius: 10px;
    }
    
    /* Selectbox */
    .stSelectbox > div > div {
        background: rgba(255, 255, 255, 0.05);
        border: 1px solid rgba(212, 165, 116, 0.3);
        border-radius: 10px;
    }
    
    /* Video container */
    .video-container {
        border: 2px solid rgba(212, 165, 116, 0.5);
        border-radius: 15px;
        overflow: hidden;
        box-shadow: 0 10px 30px rgba(0, 0, 0, 0.3);
    }
    
    /* Status messages */
    .status-box {
        padding: 1rem;
        border-radius: 10px;
        margin: 1rem 0;
    }
    
    .status-success {
        background: rgba(39, 174, 96, 0.2);
        border: 1px solid #27ae60;
    }
    
    .status-error {
        background: rgba(231, 76, 60, 0.2);
        border: 1px solid #e74c3c;
    }
    
    /* Footer */
    .footer {
        text-align: center;
        color: #666;
        padding: 2rem;
        border-top: 1px solid rgba(212, 165, 116, 0.2);
        margin-top: 3rem;
    }
</style>
""", unsafe_allow_html=True)

# Initialize session state
if 'generator' not in st.session_state:
    st.session_state.generator = None
if 'generated_videos' not in st.session_state:
    st.session_state.generated_videos = []
if 'current_video' not in st.session_state:
    st.session_state.current_video = None

# Main Title
st.markdown('<h1 class="main-title">🏍️ Vintage Bike Video Generator</h1>', unsafe_allow_html=True)
st.markdown('<p class="subtitle">Create stunning cinematic videos of classic motorcycles using AI</p>', unsafe_allow_html=True)

# Sidebar - Settings
with st.sidebar:
    st.markdown("## ⚙️ Generation Settings")
    
    # Era Selection
    era = st.selectbox(
        "🗓️ Era",
        options=list(ERA_DESCRIPTIONS.keys()),
        index=2,  # Default to 1970s
        help="Select the vintage era for your motorcycle"
    )
    
    # Camera Style
    camera = st.selectbox(
        "🎥 Camera Style",
        options=list(CAMERA_STYLES.keys()),
        index=1,  # Default to Pan
        help="Select the camera movement style"
    )
    
    # Advanced Settings
    with st.expander("🔧 Advanced Settings"):
        seed = st.number_input(
            "Seed (0 for random)",
            min_value=0,
            max_value=2**32-1,
            value=0,
            help="Set a seed for reproducible results"
        )
        
        use_cache = st.checkbox(
            "Use cached videos",
            value=True,
            help="Reuse previously generated videos with same settings"
        )
    
    st.markdown("---")
    
    # GPU Status
    st.markdown("### 💻 System Status")
    try:
        import torch
        if torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
            st.success(f"GPU: {gpu_name}")
            st.info(f"Memory: {gpu_mem:.1f} GB")
        else:
            st.error("No GPU detected!")
    except:
        st.warning("Could not detect GPU status")

# Main Content
col1, col2 = st.columns([1, 1])

with col1:
    st.markdown("### 📝 Your Prompt")
    
    # Preset Selection
    preset = st.selectbox(
        "Quick Presets",
        options=["Custom"] + list(PRESET_PROMPTS.keys()),
        help="Choose a preset or write your own"
    )
    
    # Prompt Input
    if preset != "Custom":
        default_prompt = PRESET_PROMPTS[preset]
    else:
        default_prompt = ""
    
    prompt = st.text_area(
        "Describe your vintage motorcycle scene",
        value=default_prompt,
        height=150,
        placeholder="e.g., A classic cafe racer motorcycle in a rustic garage, warm lighting, chrome details gleaming..."
    )
    
    # Additional Scene Description
    scene = st.text_input(
        "Additional scene details (optional)",
        placeholder="e.g., morning mist, autumn colors, urban setting"
    )
    
    # Generate Button
    generate_btn = st.button("🎬 Generate Video", type="primary", use_container_width=True)
    
    # Prompt Validation
    if prompt:
        is_valid, message = validate_prompt(prompt)
        if is_valid:
            st.success(message)
        else:
            st.error(message)

with col2:
    st.markdown("### 🎥 Generated Video")
    video_placeholder = st.empty()
    status_placeholder = st.empty()
    
    if st.session_state.current_video and os.path.exists(st.session_state.current_video):
        video_placeholder.video(st.session_state.current_video)

# Generation Logic
if generate_btn:
    if not prompt:
        st.error("❌ Please enter a prompt")
    else:
        is_valid, message = validate_prompt(prompt)
        
        if not is_valid:
            st.error(message)
        else:
            # Build the full prompt
            full_prompt = build_prompt(prompt, era, camera, scene)
            
            # Show the enhanced prompt
            with st.expander("📜 View Enhanced Prompt"):
                st.code(full_prompt, language=None)
            
            # Check cache
            actual_seed = seed if seed > 0 else None
            cache_key = generate_cache_key(full_prompt, era, camera, seed)
            
            if use_cache:
                cached_video = get_cached_video(cache_key)
                if cached_video:
                    st.session_state.current_video = cached_video
                    video_placeholder.video(cached_video)
                    status_placeholder.success("⚡ Served from cache!")
                    st.stop()
            
            # Generate video
            with st.spinner("🎬 Generating video... This may take 1-3 minutes"):
                progress_bar = st.progress(0)
                status_text = st.empty()
                
                try:
                    # Initialize generator if needed
                    status_text.text("Loading model...")
                    progress_bar.progress(10)
                    
                    if st.session_state.generator is None:
                        st.session_state.generator = VintageVideoGenerator()
                    
                    generator = st.session_state.generator
                    
                    status_text.text("Generating frames...")
                    progress_bar.progress(30)
                    
                    start_time = time.time()
                    frames, used_seed = generator.generate(full_prompt, actual_seed)
                    gen_time = time.time() - start_time
                    
                    status_text.text("Saving video...")
                    progress_bar.progress(80)
                    
                    # Save video
                    cache_key = generate_cache_key(full_prompt, era, camera, used_seed)
                    mp4_path, gif_path = save_video(frames, cache_key)
                    
                    progress_bar.progress(100)
                    
                    # Display result
                    st.session_state.current_video = mp4_path
                    st.session_state.generated_videos.append({
                        "path": mp4_path,
                        "prompt": prompt,
                        "seed": used_seed,
                        "time": datetime.now().strftime("%H:%M:%S")
                    })
                    
                    video_placeholder.video(mp4_path)
                    status_placeholder.success(
                        f"✅ Generated in {gen_time:.1f}s | Seed: {used_seed} | "
                        f"Frames: {len(frames)}"
                    )
                    
                    # Download buttons
                    col_dl1, col_dl2 = st.columns(2)
                    with col_dl1:
                        with open(mp4_path, "rb") as f:
                            st.download_button(
                                "📥 Download MP4",
                                f,
                                file_name=f"vintage_bike_{used_seed}.mp4",
                                mime="video/mp4"
                            )
                    with col_dl2:
                        with open(gif_path, "rb") as f:
                            st.download_button(
                                "📥 Download GIF",
                                f,
                                file_name=f"vintage_bike_{used_seed}.gif",
                                mime="image/gif"
                            )
                    
                    status_text.empty()
                    progress_bar.empty()
                    
                except Exception as e:
                    st.error(f"❌ Generation failed: {str(e)}")
                    import traceback
                    st.code(traceback.format_exc())

# History Section
st.markdown("---")
st.markdown("### 📂 Generation History")

cached_videos = list_cached_videos()
if cached_videos:
    cols = st.columns(4)
    for i, video_path in enumerate(cached_videos[:8]):
        with cols[i % 4]:
            st.video(video_path)
            if st.button(f"View", key=f"view_{i}"):
                st.session_state.current_video = video_path
                st.rerun()
else:
    st.info("No videos generated yet. Create your first vintage bike video above!")

# Footer
st.markdown("""
<div class="footer">
    <p>🏍️ Vintage Bike Video Generator | Powered by ModelScope Text-to-Video</p>
    <p>Optimized for Google Colab Free Tier (T4 GPU)</p>
</div>
""", unsafe_allow_html=True)

## 4️⃣ Setup ngrok for Public URL (Required for Colab)

In [ ]:
# Get your free ngrok authtoken from: https://dashboard.ngrok.com/get-started/your-authtoken
# Paste it below:

NGROK_AUTH_TOKEN = ""  # <-- Paste your ngrok auth token here (optional but recommended)

if NGROK_AUTH_TOKEN:
    !ngrok authtoken {NGROK_AUTH_TOKEN}
    print("✅ ngrok authenticated!")
else:
    print("⚠️ No ngrok token provided. Using localtunnel instead.")

## 5️⃣ Pre-load the Model (Optional but Recommended)

In [ ]:
# Pre-load the model to avoid timeout during first generation
from video_engine import VintageVideoGenerator

print("🔄 Pre-loading video generation model...")
generator = VintageVideoGenerator()
generator.load_model()
print("✅ Model ready! You can now run the Streamlit app.")

## 6️⃣ Launch Streamlit App

In [ ]:
# Method 1: Using ngrok (Recommended - More stable)
from pyngrok import ngrok
import subprocess
import time

# Kill any existing streamlit processes
!pkill -f streamlit

# Start Streamlit in background
process = subprocess.Popen(
    ["streamlit", "run", "streamlit_app.py", 
     "--server.port", "8501",
     "--server.headless", "true",
     "--browser.gatherUsageStats", "false"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for Streamlit to start
print("⏳ Starting Streamlit server...")
time.sleep(5)

# Create ngrok tunnel
try:
    public_url = ngrok.connect(8501)
    print("\n" + "="*60)
    print("🚀 STREAMLIT APP IS RUNNING!")
    print("="*60)
    print(f"\n🌐 Public URL: {public_url}")
    print("\n📋 Copy the URL above and open it in your browser")
    print("\n⚠️ Keep this cell running! The app will stop if you interrupt it.")
    print("="*60)
except Exception as e:
    print(f"❌ ngrok failed: {e}")
    print("\n🔄 Trying alternative method (localtunnel)...")
    !npx localtunnel --port 8501

In [ ]:
# Method 2: Using localtunnel (Alternative if ngrok doesn't work)
# Uncomment and run this cell if the above method fails

# import subprocess
# import time

# # Kill any existing streamlit processes
# !pkill -f streamlit

# # Start Streamlit
# !nohup streamlit run streamlit_app.py --server.port 8501 --server.headless true &

# time.sleep(5)
# print("Starting tunnel...")

# # Use localtunnel
# !npx localtunnel --port 8501

## 7️⃣ Quick Test (Without Streamlit)

In [ ]:
# Quick test to verify the video generation works
from video_engine import VintageVideoGenerator
from prompt_builder import build_prompt
from video_utils import save_video
from IPython.display import Video, display

# Initialize generator
gen = VintageVideoGenerator()

# Create test prompt
test_prompt = build_prompt(
    user_prompt="classic cafe racer motorcycle in garage",
    era="1970s",
    camera="Pan"
)

print(f"📝 Test Prompt: {test_prompt}\n")

# Generate
frames, seed = gen.generate(test_prompt, seed=42)

# Save
mp4_path, gif_path = save_video(frames, f"test_{seed}")

# Display
print(f"\n✅ Video saved to: {mp4_path}")
display(Video(mp4_path, embed=True, width=400))

## 📚 Usage Tips

### Best Prompts for Vintage Bikes:
- Use specific era references: "1960s British cafe racer", "1970s chopper"
- Include details: "chrome exhaust pipes", "leather seat", "spoke wheels"
- Set the scene: "in a rustic garage", "on a desert highway", "morning mist"
- Mention lighting: "golden hour", "warm workshop lighting", "dramatic shadows"

### Memory Optimization:
- The model uses ~6-8GB GPU memory
- If you get OOM errors, restart the runtime and try again
- Lower resolution (256x256) ensures it runs on T4 GPU

### Improving Quality:
- Increase `num_inference_steps` in `video_config.py` (slower but better)
- Use consistent seeds to reproduce good results
- The 1970s and Pan camera style often give best results

---

**Created for vintage motorcycle enthusiasts 🏍️**